Dask + Multi processing

In [13]:
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask_ml
import joblib
import dask
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.csvcsv)
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
# from sklearn.impute import KNNImputer
# #import matplotlib.pyplot as plot
# from matplotlib import pyplot
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import time
import warnings
warnings.filterwarnings('ignore')
import random
from xgboost import XGBClassifier
# import shutil
# import os
from our_evaluate_sepsis_score import evaluate_performance_dask
import our_functions_library as flib
from numba import jit, cuda

In [14]:
# only run this once
client = Client()

In [67]:
client

Client Scheduler: tcp://127.0.0.1:64045 Dashboard: http://127.0.0.1:64044/status,Cluster Workers: 4 Cores: 16 Memory: 16.95 GB


In [16]:
# client.shutdown()
#client.close()

 # Yeah, you may start coding here

In [101]:
# dataType={'Age': 'int', 'Gender': bool, 'Patient_id': int, 'time': 'int8', 'SepsisLabel':'float16'}
filename = '../../raw_data/raw_data_1000.csv' # use raw dataset
originalData = dd.read_csv(filename,dtype={'Gender':bool,'SepsisLabel':bool}) # read csv data into Dask DataFrame
Uniq_ID= np.unique(originalData['Patient_id'])
print("Data size:",originalData.shape)
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)

X_full_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_full_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
# X_train = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# X_test = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# y_train = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)
# y_test = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)

# Below are the lists for KNN results of different K value
KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []
KNN_F1Score_mean = []
KNN_F1Score_std = []
KNN_auroc_mean = []
KNN_auprc_mean = []
KNN_accuracy_mean = []
KNN_accuracy_std = []
KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]
fillmethod =""  

Data size: (Delayed('int-f58c8af2-8f3f-4dea-8786-b085e30f750b'), 43)
Patient id size: 1000
[   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   95   96   97   98
   99  100  101  102  103  104  105  106  107  108  109  110  111  112
  113  114  115  116  117  118  119  120  121  122  123  124  125  126
  127  128  129  130  131  132  133  134  135  136  137  138  139  140
  141  142  143  144  145  146  147  148  149  150  151  152  153  154
  155  156  157  158  159  160  161  162  163  164  165  166  167  168
  169  170  171  172  173  174  175  176  177  178  179  

In [102]:
print("Data set size:",originalData.compute().shape)
print(originalData.memory_usage(deep=True).sum().compute())

Data set size: (38890, 43)
5064968


In [113]:
missing = originalData.isna().sum()/len(originalData)*100
almostMissingColumns = list(missing[missing>90].index)
# print(almostMissingColumns)
leftAllColumns = list(originalData.columns).copy()
# print(leftAllColumns)
leftAllColumns =  [i for i in leftAllColumns if i not in almostMissingColumns]
# print(leftAllColumns)
trainColumns = leftAllColumns.copy()
# trainColumns.remove('time')
trainColumns.remove('SepsisLabel')
# print(missing.compute())
print(leftAllColumns)
print(trainColumns)

['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'Glucose', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_id', 'time']
['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'Glucose', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'Patient_id', 'time']


In [114]:
originalData = originalData[leftAllColumns]
print(originalData.head())

      HR  O2Sat   Temp    SBP     MAP  DBP  Resp  Glucose    Age  Gender  \
0   97.0   95.0  36.11   98.0  75.330  0.0  19.0    193.0  83.14   False   
1   97.0   95.0  36.11   98.0  75.330  0.0  19.0    193.0  83.14   False   
2   89.0   99.0  36.11  122.0  86.000  0.0  22.0    193.0  83.14   False   
3   90.0   95.0  36.11  122.0  88.665  0.0  30.0    193.0  83.14   False   
4  103.0   88.5  36.11  122.0  91.330  0.0  24.5    193.0  83.14   False   

   Unit1  Unit2  HospAdmTime  ICULOS  SepsisLabel  Patient_id  time  
0    0.0    0.0        -0.03       1        False           1     0  
1    0.0    0.0        -0.03       2        False           1     1  
2    0.0    0.0        -0.03       3        False           1     2  
3    0.0    0.0        -0.03       4        False           1     3  
4    0.0    0.0        -0.03       5        False           1     4  


In [115]:
#data, fillmethod = flib.linearFillingAll(originalData)#
#data = flib.linearFillingAll(originalData,1)#fill other missing data with 0
#data = flib.linearFillingAll(originalData,2,)#fill other missing data with overall average value
#data = flib.linearFillingAll(originalData,0,True)#forwardfilling
#data = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0
originalData,fillingmethod = flib.linearFillingAll(originalData.compute(),2,True)#forwardfilling + fill other missing data with overall average value
#oridata after linearly imputation is filled, so oridata needs to be generated again
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
originalData = dd.from_pandas(originalData,npartitions = 10)

filling using Forward filling
The filling choice is of number: 2
HR             0.0
O2Sat          0.0
Temp           0.0
SBP            0.0
MAP            0.0
DBP            0.0
Resp           0.0
Glucose        0.0
Age            0.0
Gender         0.0
Unit1          0.0
Unit2          0.0
HospAdmTime    0.0
ICULOS         0.0
SepsisLabel    0.0
Patient_id     0.0
time           0.0
dtype: float64
fillingChoice =2
total time: 15.25 sec


In [116]:
originalData.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_id,time
0,97.0,95.0,36.11,98.0,75.330,0.0,19.0,193.0,83.14,False,0.0,0.0,-0.03,1,False,1,0
1,97.0,95.0,36.11,98.0,75.330,0.0,19.0,193.0,83.14,False,0.0,0.0,-0.03,2,False,1,1
2,89.0,99.0,36.11,122.0,86.000,0.0,22.0,193.0,83.14,False,0.0,0.0,-0.03,3,False,1,2
3,90.0,95.0,36.11,122.0,88.665,0.0,30.0,193.0,83.14,False,0.0,0.0,-0.03,4,False,1,3
4,103.0,88.5,36.11,122.0,91.330,0.0,24.5,193.0,83.14,False,0.0,0.0,-0.03,5,False,1,4


In [117]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(range(1,21),Uniq_ID[:20]) ):
    print("The original uniq id set is:\n",Uniq_ID[0:20])
    print("The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs 
#     Uniq_ID = da.from_array(Uniq_ID)
else:
    print("They are not the same. Uniq_id has already been shuffled.")    

print("The fixed shuffelld id set, it should be 211 275 153...\n",Uniq_ID[:20])

They are not the same. Uniq_id has already been shuffled.
The fixed shuffelld id set, it should be 211 275 153...
 [291 215 113 545 697 542 794 673 660 340 299 646 822 497  93 290 207 212
 633 219]


In [118]:
def KNN_reset():
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()

def generateTrainDataSet(originalData,test_patienIds, X_columns = X_full_columns, y_columns = y_full_columns):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns] 
    return X_train, y_train


#Function to generate the test dataset
def generateTestDataSet(originalData,patienIds,X_columns = X_full_columns, y_columns = y_full_columns):
    print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test


In [119]:
#Function for 1 fold in Kfold
def OneFold_patient_MP(i,model,KforKNN,idSets,fillmethod = ""): 
    start1 = time.time()
    print("for the",i+1,"th iteration: ")
    X_test,y_test = generateTestDataSet(originalData,idSets[i],X_columns = trainColumns)
    print("Time for splitting id to test dataset:", round(time.time()- start1,3))
    t1 = time.time()
    X_train, y_train = generateTrainDataSet(originalData,idSets[i],X_columns = trainColumns)
    print("Time for splitting id to train dataset:", round(time.time()- t1,3))

    #Now the train and test dataset is generated 
    patientID_ytest = y_test['Patient_id']
    y_test = y_test.drop('Patient_id', 1) 
    y_train = y_train.drop('Patient_id', 1)
    X_train=X_train.astype('float32')
    X_test=X_test.astype('float32') 
    y_train=y_train.astype('bool')
    y_test=y_test.astype('bool')
    print(X_train)
    
    #fill the missing data
    if X_train.isnull().values.any() or X_test.isnull().values.any() :
        t1= time.time()
        print("X_train or X_test contains NaN values, KNN is performed.")
#         X_train, X_test,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)  
        X_train, X_test, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN)
        print("The type of X_train after KNN filling is",type(X_train))
        print("Time for NaN values filling: ", round(time.time()- t1,2) )
        X_train = dd.from_array(X_train)
        X_test = dd.from_array(X_test)
        if  X_train.isnull().values.any() or X_test.isnull().values.any() :
            print("X_train or X_test still contains NaN values after filling")          
    print("The type of X_train after KNN filling is",type(X_train))
    
    
    if ( type(model).__name__ == 'LogisticRegression'):  #Scale the data for logistic regression
        t1 = time.time()
        scaler = dask_ml.preprocessing.StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        print("Standard Scaling time: ", round(time.time()- t1,2) )

               
    #fit the model and predict
    with joblib.parallel_backend("dask",scatter=[X_train, y_train]):
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test).astype(bool)   
        y_predicted_probobility = model.predict_proba(X_test).astype('float32') 
    y_labels = dd.from_pandas(y_test['SepsisLabel'].compute().reset_index(drop=True),npartitions = 2)
    
#     if(i == 1):
#             # create excel writer object
#         writer = pd.ExcelWriter('X_train_90dropped_linearf.xlsx')
#         # write dataframe to excel
#         X_train.compute().to_excel(writer)
#         # save the excel
#         writer.save()
        
    del X_train, X_test, y_train

    print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
    if(len(y_predicted_probobility[0]) ==2 ):
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
    elif(len(y_predicted_probobility[0]) == 1):     
        y_predicted_probobility =  y_predicted_probobility.round(4)
    
    auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance_dask(y_labels.compute(), y_predicted, y_predicted_probobility,patientID_ytest.compute(),idSets[i]) 
    del y_labels, y_predicted_probobility    
    result = { "auroc": round(auroc,4), "auprc": round(auprc,4), "f_measure": round(f_measure,4),
                "physio_accuracy": round(physio_accuracy,4),"utility_score": round(utility_score,4), 
                "positiveprediction": np.sum(y_predicted), 
            "baseline": round( (1 - round(y_test.mean().compute().loc['SepsisLabel'],2) )*100 , 2 )}
    print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
    del  y_test
    print("Time spent in ",i,"th Fold iteration",round(time.time()-start1,2),"sec.\n")
    print("******************************************************************")
    return result

In [120]:
# Code for K-fold algorithm using multi processes
def KFold_patient_DASKMP(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes  
    print("Datasize",len(Uniq_ID) ,"with K Fold of ",KforKFold ," folds filling by KNN =",KforKNN)
    # initialisation of the array for storing the different intermediate results
    auroc_model = []
    auprc_model = []
    f_measure_model = []
    physio_accuracy_model = []
    utility_score_model = []
    positivepredictions = []
    baseline_model= []
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups

    flib.WritePatientIdsToFile(idSets, KforKFold)

#     OneFold_patient_MP(1, model, KforKNN,idSets)
    
    futures = []
    for i in range(0,KforKFold):
        print(i)
        f = client.submit(OneFold_patient_MP,i, model, KforKNN,idSets,fillmethod)
        futures.append(f)

    for f in futures:
        result = f.result()
        auroc_model.append(result["auroc"])
        auprc_model.append(result["auprc"])
        f_measure_model.append(result["f_measure"])
        physio_accuracy_model.append(result["physio_accuracy"])  
        utility_score_model.append(result["utility_score"])
        positivepredictions.append(result["positiveprediction"])
        baseline_model.append(result["baseline"])        
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('Utility score of model:', utility_score_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility accuracy of model:', physio_accuracy_model) 
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print("Positive Prediction: ", positivepredictions)
    print('Baseline model:', baseline_model)  
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_baseline_mean.append(np.mean(baseline_model))
    
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")

In [121]:
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, n_jobs=4)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [122]:
%%time
KFold_patient_DASKMP(decisionTreeModel,10,25)

Datasize 1000 with K Fold of  10  folds filling by KNN = 25
Time for writing id to file: 0.0
0
1
2
3
4
5
6
7
8
9

Evaluation parameters of the utiltiy evaluation function:
Utility score of model: [0.1098, 0.1429, 0.0748, 0.2532, 0.1712, -0.021, 0.2447, 0.2223, 0.1197, 0.2452]
utility F1 of model: [0.0787, 0.1284, 0.0979, 0.297, 0.1121, 0.0168, 0.1394, 0.1467, 0.093, 0.2277]
Utility accuracy of model: [0.9359, 0.9563, 0.9627, 0.9798, 0.9514, 0.9691, 0.9355, 0.9445, 0.8979, 0.9598]
auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Positive Prediction:  [207, 136, 77, 44, 183, 63, 228, 193, 365, 118]
Baseline model: [99.0, 98.0, 98.0, 98.0, 99.0, 99.0, 98.0, 98.0, 97.0, 98.0]

Total Time spent in  KFold function 5.28 sec.

Wall time: 5.28 s


In [29]:
y_labels

NameError: name 'y_labels' is not defined

In [ ]:
# print("X_train size is: ",X_train.size * X_train.itemsize)
# print("X_test size is: ",X_test.size * X_test.itemsize)
# print("X_train size is: ",X_train.memory_usage(deep=True).sum().compute())
# print("X_test size is: ",X_test.memory_usage(deep=True).sum().compute())
# print("y_train size is: ",y_train.memory_usage(deep=True).sum().compute())
# print("y_test size is: ",y_test.memory_usage(deep=True).sum().compute())

In [ ]:
def Print_DATA():
    print("Numbers of K:",len(KNN_accuracy_mean))
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)      

def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
    if(KforKNNend<=1):
        print("K must be a interger larger than 1")
        return
    KNN_reset()
    step=stepsize
    for i in range(KforKNNstart,KforKNNend+1,step) : 
        print("KNN of K = ",i)    
        KFold_patient(model,10,i)

In [ ]:
Print_DATA()